# Configuration files

This example shows how to use PyAEDT to export configuration files and reuse
them to import in a new project. A configuration file is supported by these applications:

* HFSS
* 2D Extractor and Q3D Extractor
* Maxwell
* Icepak (in AEDT)
* Mechanical (in AEDT)

The following topics are covered:

* Variables
* Mesh operations (except Icepak)
* Setup and optimetrics
* Material properties
* Object properties
* Boundaries and excitations

When a boundary is attached to a face, PyAEDT tries to match it with a
``FaceByPosition`` on the same object name on the target design. If for
any reason this face position has changed or the object name in the target
design has changed, the boundary fails to apply.

Keywords: **AEDT**, **general**, **configuration file**, **setup**.

## Perform imports and define constants
Import the required packages.

In [1]:
import os
import tempfile
import time

import ansys.aedt.core
from ansys.aedt.core.examples.downloads import download_icepak

Define constants.

In [2]:
AEDT_VERSION = "2025.2"
NG_MODE = False  # Open AEDT UI when it is launched.

## Create temporary directory

Create the temporary directory.
If you'd like to retrieve the project data for subsequent use,
the temporary folder name is given by ``temp_folder.name``.

In [3]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")

## Download project

Download the Icepack project.

In [4]:
project_full_name = download_icepak(
    local_path=temp_folder.name
)

## Open project

Open the Icepak project from the project folder.

In [5]:
ipk = ansys.aedt.core.Icepak(
    project=project_full_name,
    version=AEDT_VERSION,
    new_desktop=True,
    non_graphical=NG_MODE,
)
ipk.autosave_disable()

PyAEDT INFO: Parsing C:\Users\ansys\AppData\Local\Temp\tmpy96hw66b.ansys\Graphics_card.aedt.


PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Initializing new Desktop session.


PyAEDT INFO: Log on console is enabled.


...

PyAEDT INFO: Project Graphics_card has been opened.


PyAEDT INFO: Active Design set to IcepakDesign1


PyAEDT INFO: Active Design set to IcepakDesign1


PyAEDT INFO: Aedt Objects correctly read


True

## Create source blocks

Create a source block on the CPU and memories.

In [6]:
ipk.assign_solid_block(object_name="CPU", power_assignment="25W")
ipk.assign_solid_block(object_name=["MEMORY1", "MEMORY1_1"], power_assignment="5W")

PyAEDT INFO: Modeler class has been initialized! Elapsed time: 0m 1sec


PyAEDT INFO: Boundary Block Block_AKS3MV has been created.


PyAEDT INFO: Boundary Block Block_Q037HN has been created.


Block_Q037HN

## Assign boundaries

Assign the opening and grille.

In [7]:
region = ipk.modeler["Region"]
ipk.assign_openings(air_faces=region.bottom_face_x.id)
ipk.assign_grille(air_faces=region.top_face_x.id, free_area_ratio=0.8)

PyAEDT INFO: Parsing design objects. This operation can take time


PyAEDT INFO: Refreshing bodies from Object Info


PyAEDT INFO: Bodies Info Refreshed Elapsed time: 0m 0sec


PyAEDT INFO: 3D Modeler objects parsed. Elapsed time: 0m 0sec


PyAEDT INFO: Boundary Opening Opening_XZHUG3 has been created.


PyAEDT INFO: Boundary Grille Grille_KTNLRQ has been created.


Grille_KTNLRQ

## Create setup

Create the setup. Properties can be set up from the ``setup`` object
with getters and setters. They don't have to perfectly match the property
syntax.

In [8]:
setup1 = ipk.create_setup()
setup1["FlowRegime"] = "Turbulent"
setup1["Max Iterations"] = 5
setup1["Solver Type Pressure"] = "flex"
setup1["Solver Type Temperature"] = "flex"
ipk.save_project()

PyAEDT INFO: Key FlowRegime matched internal key 'Flow Regime' with confidence of 94.


PyAEDT INFO: Key Max Iterations matched internal key 'Convergence Criteria - Max Iterations' with confidence of 54.


PyAEDT INFO: Key Solver Type Pressure matched internal key 'Linear Solver Type - Pressure' with confidence of 80.


PyAEDT INFO: Key Solver Type Temperature matched internal key 'Linear Solver Type - Temperature' with confidence of 82.


PyAEDT INFO: Project Graphics_card Saved correctly


True

## Export project to step file

Export the project to the step file.

In [9]:
filename = ipk.design_name
file_path = os.path.join(ipk.working_directory, filename + ".step")
ipk.export_3d_model(
    file_name=filename,
    file_path=ipk.working_directory,
    file_format=".step",
    assignment_to_export=[],
    assignment_to_remove=[],
)

True

## Export configuration files

Export the configuration files. You can optionally disable the export and
import sections. Supported formats are JSON and TOML files.

In [10]:
conf_file = ipk.configurations.export_config(
    os.path.join(ipk.working_directory, "config.toml")
)
ipk.close_project()

PyAEDT INFO: Parsing C:\Users\ansys\AppData\Local\Temp\tmpy96hw66b.ansys\Graphics_card.aedt.


PyAEDT INFO: File C:\Users\ansys\AppData\Local\Temp\tmpy96hw66b.ansys\Graphics_card.aedt correctly loaded. Elapsed time: 0m 0sec


PyAEDT INFO: aedt file load time 0.04738116264343262


PyAEDT WARNING: Property Command is read-only.


PyAEDT INFO: Mesh class has been initialized! Elapsed time: 0m 0sec


...

PyAEDT INFO: C:\Users\ansys\AppData\Local\Temp\tmpy96hw66b.ansys\Graphics_card.pyaedt\IcepakDesign1\config.toml correctly created.


PyAEDT INFO: Json file C:\Users\ansys\AppData\Local\Temp\tmpy96hw66b.ansys\Graphics_card.pyaedt\IcepakDesign1\config.toml created correctly.


PyAEDT INFO: Closing the AEDT Project Graphics_card


PyAEDT INFO: Project Graphics_card closed correctly


True

## Create project

Create an Icepak project and import the step.

In [11]:
new_project = os.path.join(temp_folder.name, "example.aedt")
app = ansys.aedt.core.Icepak(version=AEDT_VERSION, project=new_project)
app.modeler.import_3d_cad(file_path)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Returning found Desktop session with PID 1392!


PyAEDT INFO: Project example has been created.


PyAEDT INFO: No design is present. Inserting a new design.


PyAEDT INFO: Added design 'Icepak_RB9' of type Icepak.


PyAEDT INFO: Aedt Objects correctly read


PyAEDT INFO: Modeler class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Step file C:\Users\ansys\AppData\Local\Temp\tmpy96hw66b.ansys\Graphics_card.pyaedt\IcepakDesign1\IcepakDesign1.step imported


True

## Import and apply configuration file

Import and apply the configuration file. You can apply all or part of the
JSON file that you import using options in the ``configurations`` object.

In [12]:
out = app.configurations.import_config(conf_file)
is_conf_imported = app.configurations.results.global_import_success

PyAEDT INFO: Parsing C:\Users\ansys\AppData\Local\Temp\tmpy96hw66b.ansys\example.aedt.


PyAEDT INFO: File C:\Users\ansys\AppData\Local\Temp\tmpy96hw66b.ansys\example.aedt correctly loaded. Elapsed time: 0m 0sec


PyAEDT INFO: aedt file load time 0.01599574089050293


PyAEDT INFO: Dataset Ceramic_material_39 doesn't exist.


PyAEDT INFO: Dataset $Ceramic_material_39 created successfully.


...

PyAEDT INFO: Boundary Operation Block_AKS3MV added.


PyAEDT INFO: Boundary Operation Block_Q037HN added.


PyAEDT INFO: Boundary Operation Opening_XZHUG3 added.


PyAEDT INFO: Boundary Operation Grille_KTNLRQ added.


PyAEDT INFO: Setup Setup added.


## Release AEDT
Close the project and release AEDT.

In [13]:
app.release_desktop()
time.sleep(3)  # Allow AEDT to shut down before cleaning the temporary project folder.

PyAEDT INFO: Desktop has been released and closed.


## Clean up

All project files are saved in the folder ``temp_folder.name``.
If you've run this example as a Jupyter notebook, you
can retrieve those project files. The following cell removes
all temporary files, including the project folder.

In [14]:
temp_folder.cleanup()